Tento jupyter ntb file sluzil pri pochopeni modelu, dat a nasledne vytvorenie funkcie na predspracovanie datasetu na pozadovany format

In [50]:
import pandas as pd
import joblib

In [51]:
model = joblib.load("model.joblib")
print(model.feature_names_in_)
print(len(model.feature_names_in_))

['longitude' 'latitude' 'housing_median_age' 'total_rooms'
 'total_bedrooms' 'population' 'households' 'median_income'
 'ocean_proximity_<1H OCEAN' 'ocean_proximity_INLAND'
 'ocean_proximity_ISLAND' 'ocean_proximity_NEAR BAY'
 'ocean_proximity_NEAR OCEAN']
13


In [52]:
file_path = "housing.csv"
df = pd.read_csv(file_path)
df.head(5)

,LONGITUDE,LAT,MEDIAN_AGE,ROOMS,BEDROOMS,POP,HOUSEHOLDS,MEDIAN_INCOME,MEDIAN_HOUSE_VALUE,OCEAN_PROXIMITY,AGENCY
0,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,YES
1,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,YES
2,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,YES
3,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,YES
4,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY,YES


In [ ]:

# AGENCY column sa medzi fitoch nenachadza v modeli
df = df.drop(columns=['AGENCY'])

print("NULL values remaining:\n", df.isnull().sum()[df.isnull().sum() > 0])

# Osetrenie NaN hodnot a dopocitanie prostrednych hodnot
df.fillna(df.median(numeric_only=True), inplace=True)

# V datasete sa nachadzaju 'Null' stringy. Nahradenie za numeric NaN
df.replace('Null', pd.NA, inplace=True)

# Konverzia vsetkych columns okrem ocean_proximity na numeric
df[df.columns[:-1]] = df[df.columns[:-1]].apply(pd.to_numeric, errors='coerce')

# Dopocitanie prostrednych hodnot pre zvysne NaN hodnoty
df.fillna(df.median(numeric_only=True), inplace=True)

# Odstranenie 1 NaN riadku
df.dropna(subset=['OCEAN_PROXIMITY'], inplace=True)

# Print remaining NULL values (if any)
print("NULL values remaining:\n", df.isnull().sum()[df.isnull().sum() > 0])


# Premenovanie columns kvoli column names uz v nafitovanom modeli
df = df.rename(columns={
    'LONGITUDE': 'longitude',
    'LAT': 'latitude',
    'MEDIAN_AGE': 'housing_median_age',
    'ROOMS': 'total_rooms',
    'BEDROOMS': 'total_bedrooms',
    'POP': 'population',
    'HOUSEHOLDS': 'households',
    'MEDIAN_INCOME': 'median_income',
    'OCEAN_PROXIMITY': 'ocean_proximity',
    'MEDIAN_HOUSE_VALUE': 'median_house_value',
})
print(df.columns)

# Kategoricky label stlpec encodeneme na 0,1
df = pd.get_dummies(df, columns=['ocean_proximity'], prefix='ocean_proximity')
expected_ocean_columns = [
    'ocean_proximity_<1H OCEAN',
    'ocean_proximity_INLAND',
    'ocean_proximity_ISLAND',
    'ocean_proximity_NEAR BAY',
    'ocean_proximity_NEAR OCEAN'
]

for col in expected_ocean_columns:
    if col not in df.columns:
        df[col] = 0


# Finalny df a zoradene stlpce X + Y (posledny stlpec)
preprocessed_df = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'] + expected_ocean_columns + ['median_house_value']]
print(preprocessed_df.columns)


preprocessed_df.to_csv("housing_preprocessed.csv", index=False)


NULL values remaining:
 MEDIAN_AGE              1
ROOMS                   2
BEDROOMS              212
POP                     1
HOUSEHOLDS              2
MEDIAN_INCOME           2
MEDIAN_HOUSE_VALUE      4
dtype: int64
NULL values remaining:
 OCEAN_PROXIMITY    1
dtype: int64
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity_<1H OCEAN', 'ocean_proximity_INLAND',
       'ocean_proximity_ISLAND', 'ocean_proximity_NEAR BAY',
       'ocean_proximity_NEAR OCEAN', 'median_house_value'],
      dtype='object')
